<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [1]</a>'.</span>

# Mobile Get Input Notebook - Phase 2: Batch Point Generation

**Phase 2**: Generate all receiver points at once using radial distribution.

This phase creates uniformly distributed receiver points around the transmitter at multiple distances and azimuths, outputting a GeoDataFrame ready for Phase 3.

## Prerequisites
- Run **Phase 0** first (phase0_setup.ipynb)
- Run **Phase 1** first (phase1_data_prep.ipynb) - optional but recommended

## Workflow
1. **Import Phase 0 outputs**: CONFIG, transmitter, paths
2. **Generate receivers**: Create points at all distances × all azimuths
3. **GeoDataFrame**: Store geometry + metadata (distance, azimuth)
4. **Output**: GeoDataFrame with ~13k points ready for Phase 3

## Output
- GeoDataFrame with receiver points (geometry + distance + azimuth)
- ~13,000 points (22 distances × 36 azimuths + 1 transmitter point)
- Stored in `receivers_gdf` variable for Phase 3

## Setup: Import from Phase 0

This cell imports all the setup from Phase 0.

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [1]:
# Import everything from Phase 0 using %run magic
%run phase0_setup.ipynb

print("\n✓ Phase 0 setup imported successfully")
print(f"  Transmitter: {tx}")
print(f"  Project root: {project_root}")

Exception: File `'phase0_setup.ipynb'` not found.

## Helper Function: Generate Receivers

Create uniformly distributed receiver points at multiple distances and azimuths.

In [ ]:
def generate_receivers_radial_multi(
    tx,
    distances_km,
    azimuths_deg,
    include_tx_point=False,
):
    """Generate receivers on multiple rings around transmitter.
    
    Args:
        tx: Transmitter object with lon, lat, tx_id
        distances_km: Array/list of distances in km
        azimuths_deg: Array/list of azimuths in degrees (0-360)
        include_tx_point: If True, include transmitter as rx_id=0
    
    Returns:
        GeoDataFrame with columns: tx_id, rx_id, distance_km, azimuth_deg, geometry
    """
    # Create transmitter point and get UTM CRS
    tx_gdf = gpd.GeoDataFrame(
        {"tx_id": [tx.tx_id]},
        geometry=[Point(tx.lon, tx.lat)],
        crs="EPSG:4326",
    )
    utm_crs = tx_gdf.estimate_utm_crs()
    tx_utm = tx_gdf.to_crs(utm_crs)
    tx_pt = tx_utm.geometry.iloc[0]

    rows = []
    rx_id = 1

    # Optional: add transmitter point at distance=0
    if include_tx_point:
        rows.append({
            "tx_id": tx.tx_id,
            "rx_id": 0,
            "distance_km": 0.0,
            "geometry": Point(tx.lon, tx.lat),
        })

    # Generate receivers at each distance × azimuth combination
    for d_km in distances_km:
        radius_m = float(d_km) * 1000.0
        for az in azimuths_deg:
            theta = math.radians(float(az))
            dx = radius_m * math.sin(theta)
            dy = radius_m * math.cos(theta)
            rx_utm = Point(tx_pt.x + dx, tx_pt.y + dy)
            
            # Convert back to WGS84 (EPSG:4326)
            rx_ll = gpd.GeoSeries([rx_utm], crs=utm_crs).to_crs("EPSG:4326").iloc[0]
            
            rows.append({
                "tx_id": tx.tx_id,
                "rx_id": rx_id,
                "distance_km": float(d_km),
                "azimuth_deg": float(az),
                "geometry": rx_ll,
            })
            rx_id += 1

    return gpd.GeoDataFrame(rows, geometry="geometry", crs="EPSG:4326")


print("✓ generate_receivers_radial_multi() defined")

## Generate All Receiver Points

Create all points at once (batch mode).

In [ ]:
print("\n" + "="*60)
print("PHASE 2: BATCH POINT GENERATION")
print("="*60)

print(f"\nGenerating receiver points:")
print(f"  Transmitter: ({tx.lat}, {tx.lon})")
print(f"  Max distance: {max_distance_km} km")
print(f"  Distances: {len(distances)} points @ {CONFIG['RECEIVER_GENERATION']['distance_step']} km spacing")
print(f"  Azimuths: {len(azimuths)} angles @ {CONFIG['RECEIVER_GENERATION']['azimuth_step']}° spacing")
print(f"  Expected points: ~{len(distances) * len(azimuths) + 1}")

# Generate all receiver points at once
start = time.time()
receivers_gdf = generate_receivers_radial_multi(
    tx,
    distances,
    azimuths,
    include_tx_point=True
)
elapsed = time.time() - start

print(f"\n✓ Generated {len(receivers_gdf)} receiver points in {elapsed:.3f}s")
print(f"\nGeoDataFrame structure:")
print(f"  Columns: {list(receivers_gdf.columns)}")
print(f"  CRS: {receivers_gdf.crs}")
print(f"\nFirst 5 points:")
print(receivers_gdf.head())
print(f"\nLast 5 points:")
print(receivers_gdf.tail())

## Summary

**Phase 2 Complete**:
- ✓ All receiver points generated in batch mode
- ✓ GeoDataFrame with geometry, distance, azimuth metadata
- ✓ Ready for Phase 3 (batch data extraction)

**Output**: `receivers_gdf` variable with ~13k points ready for Phase 3

**Next**: Run Phase 3 to extract elevation and land cover data for all points at once